In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from groq import Groq
from time import sleep
import re
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,accuracy_score
import random
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# Make a Function to Debug and get the result of the Model or The Technique
def dprint(s, debug):
    if debug:
        print(s)

In [ ]:

YOUR_GROQ_API_KEY = ''  # Get from https://console.groq.com/keys
groq_client = Groq(api_key=YOUR_GROQ_API_KEY)

def call_groq_api(prompt, student_configs, pre_processing, post_processing, model='llama3-70b-8192', debug=False):
    prompt = pre_processing(prompt)
    # Initlize the defualt Parameters or Configrations for Groq API 
    groq_params = {
        'messages': [{'role': 'user', 'content': prompt}],
        'model': model,
        'max_tokens': student_configs.get('max_tokens', 512),
        'temperature': student_configs.get('temperature', 0.7),
        'top_p': student_configs.get('top_p', 0.7),
        'stop': student_configs.get('stop', None),
    }
    
    output = groq_client.chat.completions.create(**groq_params)
    
    dprint('*****prompt*****', debug)
    dprint(prompt, debug)
    dprint('*****result*****', debug)
    res = output.choices[0].message.content
    dprint(res, debug)
    dprint('*****output*****', debug)
    labels_only = post_processing(res)
    dprint('POST PROCESSED', debug)
    dprint(labels_only, debug)
    dprint('=========', debug)
    return labels_only

In [ ]:
# Most likely current Groq model names:
model_names = [
    'llama-3.1-8b-instant',     
    "llama-3.3-70b-versatile",  # The Model We Use            
]

In [ ]:
def get_train_df(topn = 10):
    train_df = pd.read_excel('train_split.xlsx')
    return train_df.sampl(topn)

In [ ]:
def get_test_set(topn=10):
    test_df = pd.read_excel('test_split.xlsx')
    return test_df.sample(topn)

In [ ]:
def get_eval_df(topn = 5):
    eval_df = pd.read_excel('dev_split.xlsx')
    return eval_df.sample(topn)

In [ ]:
# Test Model Function with the the Data Labels and Model Answers and prepare Data Frame for Report 
def test_range(df, prompt_configs, prompt_prefix, examples, prompt_suffix,
               pre_processing=lambda x:x, post_processing=lambda y:y,
               model='llama-3.3-70b-versatile', debug=False):
    text_ids = []
    answers = []
    model_responses = []
    corrected_model_responses = []
    text_list = []
    
    for idx, row in tqdm(df.iterrows()):
        text_ids.append(idx)  
        fixed_prompt = row['text'] + "\n"
        text_list.append(row['text'])
        fixed_prompt = pre_processing(fixed_prompt)
        prompt = prompt_prefix + examples + fixed_prompt + prompt_suffix
        answer = row['label']
        answers.append(answer)
        model_response = call_groq_api(prompt, prompt_configs, pre_processing, lambda y:y, model=model, debug=debug) 
        corrected_model_response = post_processing(model_response)
        corrected_model_responses.append(corrected_model_response)
        model_responses.append(model_response)
        sleep(1)
    
    result_df = pd.DataFrame({
        'text_id': text_ids, 
        'text': text_list, 
        'model_responses': model_responses, 
        'corrected_model_responses': corrected_model_responses, 
        'true_label': answers
    })
    return result_df

In [ ]:
# Get The Data With The Number or Rows we will test on it from Evaloation Data
data=get_eval_df(50)

## Zero Shot Prompt

In [ ]:
# Get the categories in The Data
def get_all_categories():
    train_df = pd.read_excel('train_split.xlsx')
    
    all_categories = sorted(train_df['label'].unique())
    print("Found categories:", all_categories)
    print("Total categories:", len(all_categories))
    
    return all_categories

categories = get_all_categories()

In [ ]:
# Prepare The Prompt For Zero Shot 
categories_list = get_all_categories()
categories_text = ", ".join(categories_list)

prompt_prefix_zs = f'''
Classify the following Combiend Arabic and English text into one of these categories: 
{categories_text}.

Your output should only be one of these exact category names:
{chr(10).join(categories_list)}
'''

prompt_examples_zs = "Input Text: "
prompt_suffix_zs = "Output: "


In [ ]:
# Pre and Post Processing for The Output 
def your_pre_processing_zs(input_string):
    return re.sub(r"@user","", input_string).strip()

def your_post_processing_zs(output_string):
    output_clean = output_string.strip().lower()
    
    if 'business' in output_clean:
        return 'business'
    elif 'shopping' in output_clean:
        return 'shopping'
    elif 'finance' in output_clean:
        return 'finance'
    elif 'education' in output_clean:
        return 'education'
    elif 'tech' in output_clean :
        return 'tech'
    elif 'technology' in output_clean or "tech" in output_clean:
        return 'tech'
    elif 'sports' in output_clean:
        return 'sports'
    elif 'medical' in output_clean :
        return 'medical'
    elif  'social' in output_clean:
        return 'social'
    else:
        return output_string.strip()  

In [ ]:
# Configration for Zeroshot Technique
prompt_config_zs = {
    'max_tokens': 3,
    'temperature': 0.4,
    'top_p': 0.7,
    'stop': []
}

model = 'llama-3.3-70b-versatile'
print(model)


In [ ]:
# Test The Technique
results_df = test_range(data, prompt_config_zs, prompt_examples_zs, prompt_prefix_zs, prompt_suffix_zs, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
print(results_df)

In [ ]:
f1_score(results_df['true_label'], results_df['corrected_model_responses'],average='weighted')

In [ ]:
print(classification_report(results_df['true_label'], results_df['corrected_model_responses'], digits=4))

## Few Shot Prompting (In Context Learning)
Useful to fix output format

In [ ]:
train_df = get_train_df(50)

In [ ]:
# Create Examples Data From Our Orignal Data
def create_example(row):
    line1 = "Input Text: " + row['text'] + "\n"
    label = row['label']  
    line2 = "Output: " + label + "\n"
    return line1 + line2

In [ ]:
prompt_examples_icl = ""
for idx,row in train_df.iterrows():
    ex = create_example(row)
    prompt_examples_icl += ex
prompt_examples_icl = prompt_examples_icl + "Input Text: "

In [ ]:
# Prepare The Prompt For Few Shot 
categories = get_all_categories()
categories_text = ", ".join(categories)

prompt_prefix_icl = f'''
Classify the following Combined Arabic and English text into one of these categories: 
{categories_text}.

Your output should only be one of these exact category names like in the examples below.
'''

prompt_suffix_icl = "Output: "


In [ ]:
prompt_config_fs = {
    'max_tokens': 3,
    'temperature': 0.4,
    'top_p': 0.7,
    'stop': []
}

In [ ]:
results_df = test_range(data, prompt_config_fs, prompt_examples_icl, prompt_prefix_icl, prompt_suffix_icl, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
print(results_df)

In [ ]:
f1_score(results_df['true_label'], results_df['corrected_model_responses'],average='weighted')

In [ ]:
print(classification_report(results_df['true_label'], results_df['corrected_model_responses'], digits=4))

## Chain of Thought Prompting

In [ ]:
# Text And Justification for each Category in The Data Manual Written
business_texts = [
    "تطوير business strategy فعاله يعتمد علي فهم market trends بشكل شامل ودقيق",
    "الشركات الناجحة تضع خطط استراتيجية واضحة لتحقيق اهدافها في السوق"
]
justification_business = [
    "This text is about business because it discusses company strategies, market analysis, and commercial planning",
    "This belongs to business category as it focuses on organizational management and market competition"
]

education_texts = [
    "في حال تم تطبيق effective curriculum development سيزيد ذلك من مستوي student engagement في الفصول الدراسيه",
    "استخدام virtual classrooms يعزز فرص collaboration بين الطلاب من مختلف انحاء العالم"
]
justification_education = [
    "This text is about education because it mentions curriculum development and student engagement in classrooms",
    "This belongs to education category as it discusses virtual learning and student collaboration"
]

finance_texts = [
    "اذا كنت تريد النجاح في trading عليك ان تتجنب emotional decisions وتركز علي market analysis",
    "تعتبر fintech من المجالات الرايده حيث تقدم حلولا مبتكره لتحسين banking services"
]
justification_finance = [
    "This text is about finance because it covers trading strategies and market analysis",
    "This belongs to finance category as it discusses fintech innovations and banking services"
]

health_texts = [
    "اذا كنت ترغب في تحسين mental health يجب عليك ممارسه mindfulness بانتظام وتناول طعام صحي",
    "ممارسة الرياضة اليومية تساعد في الحفاظ على صحة القلب والجسم بشكل عام"
]
justification_health = [
    "This text is about health because it discusses mental health, mindfulness, and healthy eating",
    "This belongs to health category as it covers physical exercise and heart health"
]

medical_texts = [
    "تعتبر nutrition السليمه اساسا لنجاح اي medical treatment لذا يجب التركيز علي الاطعمه الغنيه بالفيتامينات",
    "هل يمكن ان توثر pharmaceutical advancements علي نسبه الشفاء من الامراض المزمنه بشكل كبير"
]
justification_medical = [
    "This text is about medical because it discusses nutrition in medical treatment and vitamins",
    "This belongs to medical category as it covers pharmaceutical advancements and chronic disease treatment"
]

shopping_texts = [
    "اذا كنت ترغب في شراء consumer electronics فمن الافضل دايما مراجعه customer reviews قبل اتخاذ قرارك",
    "مقارنة الاسعار بين المتاجر المختلفة تساعد في اتخاذ قرار شراء أفضل"
]
justification_shopping = [
    "This text is about shopping because it mentions consumer electronics and customer reviews",
    "This belongs to shopping category as it discusses price comparison and purchase decisions"
]

social_texts = [
    "تفاعل الافراد في المجتمع يساهم في بناء علاقات قوية وتحسين جودة الحياة الاجتماعية",
    "المبادرات المجتمعية تلعب دوراً هاماً في حل المشكلات الاجتماعية وتعزيز التكافل"
]
justification_social = [
    "This text is about social because it discusses community interactions and social relationships",
    "This belongs to social category as it covers community initiatives and social problem-solving"
]

sports_texts = [
    "اللاعبون المميزون في cricket يعرفون كيف يستخدمون strategic planning للفوز بالمباريات المهمه",
    "التدريب المستمر والالتزام بالبرنامج الرياضي اساسي لتحقيق النتائج في المسابقات"
]
justification_sports = [
    "This text is about sports because it mentions cricket players and strategic planning in games",
    "This belongs to sports category as it discusses continuous training and sports competitions"
]

tech_texts = [
    "اذا استثمرت الشركات في cloud computing ستتمكن من تحسين كفاءه عملياتها وتقليل التكاليف",
    "الذكاء الاصطناعي يساهم في تطوير قطاعات متعددة من الصناعة والخدمات"
]
justification_tech = [
    "This text is about tech because it covers cloud computing and operational efficiency",
    "This belongs to tech category as it discusses artificial intelligence and industry development"
]

In [ ]:
# Create Examples from Text And Justifications
def create_examples(all_texts, all_justifications, all_labels):
    examples = []

    combined_data = list(zip(all_texts, all_justifications, all_labels))
    random.shuffle(combined_data)

    for text, justification, label in combined_data:
        line1 = "Input Text: " + text + "\n"
        justification_line = "Justification: " + justification + "\n"
        line2 = "Output: " + label + "\n"
        examples.append(line1 + justification_line + line2)

    return ''.join(examples)

In [ ]:
#Combine The Text And The Justification 
all_topic_texts = (business_texts + education_texts + finance_texts + 
                   health_texts + medical_texts + shopping_texts + 
                   social_texts + sports_texts + tech_texts)

all_topic_justifications = (justification_business + justification_education + justification_finance +
                           justification_health + justification_medical + justification_shopping +
                           justification_social + justification_sports + justification_tech)

all_topic_labels = (['business'] * len(business_texts) + 
                   ['education'] * len(education_texts) +
                   ['finance'] * len(finance_texts) +
                   ['health'] * len(health_texts) +
                   ['medical'] * len(medical_texts) +
                   ['shopping'] * len(shopping_texts) +
                   ['social'] * len(social_texts) +
                   ['sports'] * len(sports_texts) +
                   ['tech'] * len(tech_texts))

In [ ]:
prompt_examples_cot = create_examples(all_topic_texts, all_topic_justifications, all_topic_labels) + "Input Text: "

In [ ]:
print(prompt_examples_cot)

In [ ]:
## Prepare The Prompt For Chain of Thoughts 

prompt_prefix_cot = f'''
Classify the following Combined Arabic and English text into one of these categories: 
{", ".join(categories)}.

Analyze the text and provide a justification before giving the final category.
Your output should include a justification and then the category name like in the examples below.
'''

prompt_suffix_cot = "Justification: "

In [ ]:
# pre and post Processing for Chain of Thoughts
def your_pre_processing_cot(input_string):
    return input_string.strip()  

def your_post_processing_cot(output_string):
    categories = ['business', 'education', 'finance', 'health', 'medical', 'shopping', 'social', 'sports', 'tech']
    output_lower = output_string.lower()
    
    for category in categories:
        if category in output_lower:
            return category
    
    return output_string.split()[0] if output_string.strip() else 'unknown'

In [ ]:
# Chain of Thoughts Technique Configration
prompt_config_cot = {
    'max_tokens': 80,  
    'temperature': 0.2, 
    'top_k': 0,       
    'top_p': 0.9,
    'repetition_penalty': 1,
    'stop': ['\nCategory:'] 
}
model = 'llama-3.3-70b-versatile'
print(model)


In [ ]:
results_df = test_range(data, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)
print(results_df)

In [ ]:
f1_score(results_df['true_label'], results_df['corrected_model_responses'],average='weighted')

In [ ]:
print(classification_report(results_df['true_label'], results_df['corrected_model_responses']))

## CARP

In [ ]:
# Prepare the featuers or key words for each Category
category_features = {
    'business': ['company strategies', 'market analysis', 'commercial planning', 'business growth', 'market trends'],
    'education': ['learning', 'teaching', 'curriculum', 'students', 'school', 'education', 'classroom', 'virtual learning'],
    'finance': ['money', 'investment', 'trading', 'banking', 'financial', 'markets', 'fintech', 'economic'],
    'health': ['fitness', 'wellness', 'mental health', 'exercise', 'healthy lifestyle', 'physical health'],
    'medical': ['treatment', 'medicine', 'pharmaceutical', 'medical care', 'doctors', 'nutrition', 'vitamins'],
    'shopping': ['purchase', 'buying', 'products', 'reviews', 'price comparison', 'consumer', 'customer'],
    'social': ['community', 'relationships', 'social interactions', 'society', 'social life', 'community initiatives'],
    'sports': ['athletes', 'games', 'training', 'competitions', 'sports', 'cricket', 'players', 'matches'],
    'tech': ['technology', 'software', 'computers', 'AI', 'digital', 'innovation', 'cloud computing', 'artificial intelligence']
}

In [ ]:
# Create Examples Prompt to Show The keyword for each category to the model
def create_carp_examples_format(all_texts, all_justifications, all_labels, category_features):

    examples = []
    combined_data = list(zip(all_texts, all_justifications, all_labels))
    
    example_subset = combined_data[:6] 
    
    for text, justification, label in example_subset:
        features = category_features.get(label, [])
        
        carp_justification = f"{justification} CARP Reasoning: Text contains features like {', '.join(features[:2])} confirming {label} category."
        
        line1 = "Input Text: " + text + "\n"
        justification_line = "Justification: " + carp_justification + "\n"
        line2 = "Output: " + label + "\n"
        examples.append(line1 + justification_line + line2)

    return ''.join(examples)

prompt_examples_carp = create_carp_examples_format(all_topic_texts, all_topic_justifications, all_topic_labels, category_features)

In [ ]:
# The prompt for CARP
prompt_prefix_carp = f"""
Classify the following Combined Arabic and English text into one of these categories:
{", ".join(category_features.keys())}.

Use the CARP method internally (analyze topic, list expected features, verify features, decide), 
but **do NOT show your reasoning**.

Output format: Only write the final category name with no explanation, no extra text, and no punctuation.
"""
prompt_suffix_carp = "Justification: "

In [ ]:
# pre and post processing for CARP
def your_pre_processing_carp(input_string):
    return input_string.strip()

def your_post_processing_carp(output_string):

    categories = list(category_features.keys())
    output_lower = output_string.lower()
    
    if "output:" in output_lower:
        output_part = output_lower.split("output:")[1].strip()
        for category in categories:
            if category in output_part.split()[0]:  
                return category
    
    for category in categories:
        if f" {category} " in f" {output_lower} ":
            return category
    
    return output_string.split()[0] if output_string.strip() else 'unknown'

In [ ]:
# CARP Configrations
prompt_config_carp = {
    'max_tokens': 150,  
    'temperature': 0.1,  
    'top_k': 0,       
    'top_p': 0.9,
    'repetition_penalty': 1.1, 
    'stop': ['\n---', 'Input Text:']
}

In [ ]:
results_df = test_range(data, prompt_config_carp, prompt_examples_carp, prompt_prefix_carp, prompt_suffix_carp, pre_processing=your_pre_processing_carp, post_processing=your_post_processing_carp, model=model, debug=False)
print(results_df)

In [ ]:
f1_score(results_df['true_label'], results_df['corrected_model_responses'],average='weighted')

In [ ]:
print(classification_report(results_df['true_label'], results_df['corrected_model_responses']))

# self consistency

In [ ]:
# Majority Vote Function To Get The Counter for each predict output for each COT and decide the final answer
def majority_vote(predictions):
    from collections import Counter
    
    if not predictions:
        return 'unknown'    
    prediction_counts = Counter(predictions)
    most_common = prediction_counts.most_common()
    
    if len(most_common) == 1:
        return most_common[0][0]
    else:
        return most_common[0][0]
# pre procissing for Self Consistency
def self_consistency_post_processing(output_string):

    categories = ['business', 'education', 'finance', 'health', 'medical', 
                 'shopping', 'social', 'sports', 'tech']
    
    output_lower = output_string.lower()
    final_answer_patterns = [
        'final answer:',
        'answer:',
        'category:',
        'classification:'
    ]
    
    for pattern in final_answer_patterns:
        if pattern in output_lower:
            parts = output_lower.split(pattern, 1)
            if len(parts) > 1:
                answer_text = parts[1].strip()
                for category in categories:
                    if category in answer_text.split()[0] if answer_text else False:
                        return category
    
    for category in categories:
        if f" {category} " in f" {output_lower} ":
            return category
    
    return output_string.split()[0] if output_string.strip() else 'unknown'

In [ ]:
# Prepare The Prompt way for Self Consistency The way of Output and the Defualt Conigrations and API Parameters
# and prepare The numbers of samples of tries with different COT 
def self_consistency_prompting(df, prompt_config, prompt_prefix, examples, prompt_suffix,
                             pre_processing=lambda x:x, post_processing=lambda y:y,
                             model='llama-3.3-70b-versatile', 
                             num_samples=3, 
                             debug=False):

    
    text_ids = []
    answers = []
    all_model_responses = []  
    final_predictions = []    
    text_list = []
    individual_predictions_list = []
    
    for idx, row in tqdm(df.iterrows()):
        text_ids.append(idx)
        fixed_prompt = row['text'] + "\n"
        text_list.append(row['text'])
        fixed_prompt = pre_processing(fixed_prompt)
        
        reasoning_prompt = prompt_prefix + examples + fixed_prompt + prompt_suffix
        
        answer = row['label']
        answers.append(answer)
        
        reasoning_samples = []
        predictions = []
        
        for sample_num in range(num_samples):
            sample_config = prompt_config.copy()
            sample_config['temperature'] = min(0.8, prompt_config.get('temperature', 0.7) + 0.1 * sample_num)
            
            model_response = call_groq_api(
                reasoning_prompt, 
                sample_config, 
                pre_processing, 
                lambda y: y, 
                model=model, 
                debug=debug
            )
            
            reasoning_samples.append(model_response)
            
            prediction = post_processing(model_response)
            predictions.append(prediction)
            
            sleep(0.5)  
        
        final_prediction = majority_vote(predictions)
        all_model_responses.append(reasoning_samples)
        individual_predictions_list.append(predictions)
        final_predictions.append(final_prediction)
        
        sleep(1)  
    
    result_df = pd.DataFrame({
        'text_id': text_ids, 
        'text': text_list, 
        'all_reasoning_samples': all_model_responses,  
        'individual_predictions': individual_predictions_list,
        'corrected_model_responses': final_predictions, 
        'true_label': answers
    })
    return result_df

In [ ]:
# Prompt for Self Consistency
def create_self_consistency_prompt():

    categories = get_all_categories()
    categories_text = ", ".join(categories)
    
    prompt_prefix_sc = f"""
Classify the following Combined Arabic and English text into one of these categories:
{categories_text}.

Analyze the text and decide the most suitable category.

Output format: Only write the final category name with no explanation, no extra text, and no punctuation.
"""

    
    examples_sc = '''
Example 1:
Input Text: تطوير business strategy فعاله يعتمد علي فهم market trends بشكل شامل
Reasoning: This text discusses business strategy development and market trends analysis, which are core business concepts. The keywords "business strategy" and "market trends" clearly indicate this belongs to business category.
Final Answer: business

Example 2:
Input Text: اذا كنت ترغب في تحسين mental health يجب عليك ممارسه mindfulness بانتظام
Reasoning: The text focuses on mental health improvement through mindfulness practice. Keywords like "mental health" and "mindfulness" are health-related topics about psychological well-being.
Final Answer: health

Example 3:
Input Text: استخدام machine learning في data analysis يحسن دقة التنبؤات
Reasoning: This text discusses machine learning and data analysis for improving prediction accuracy. These are technical concepts related to technology and artificial intelligence.
Final Answer: tech
'''
    
    prompt_suffix_sc = "Reasoning: "
    
    return prompt_prefix_sc, examples_sc, prompt_suffix_sc

prompt_config_sc = {
    'max_tokens': 150, 
    'temperature': 0.7,  
    'top_p': 0.9,
    'stop': ['\nInput Text:', 'Example:']  # Stop conditions
}

In [ ]:
# Test The Model 
prompt_prefix_sc, prompt_examples_sc, prompt_suffix_sc = create_self_consistency_prompt()
results_df = self_consistency_prompting(
    data, 
    prompt_config_sc, 
    prompt_examples_sc, 
    prompt_prefix_sc, 
    prompt_suffix_sc,
    pre_processing=your_pre_processing_zs,
    post_processing=self_consistency_post_processing,
    model=model,
    num_samples=3, 
    debug=False
)

print(results_df[['text_id', 'text', 'corrected_model_responses', 'true_label']])

In [ ]:
f1_score(results_df['true_label'], results_df['corrected_model_responses'],average='weighted')

In [ ]:
print(classification_report(results_df['true_label'], results_df['corrected_model_responses']))

# RAG

In [ ]:
# Config The Model as a Client waiting for prompt
client = Groq(api_key="")
def chat_completion(prompt):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [ ]:
# Load The data
df = pd.read_excel("train_split.xlsx")
df["combined"] = df.astype(str).agg(" ".join, axis=1)
texts = df["text"].astype(str).tolist()
labels = df["label"].astype(str).tolist()
print(f"Loaded {len(df)} samples.")

In [ ]:
# Transform The Date from Sentence to vector to can Handle it 

# model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = model.encode(texts, convert_to_numpy=True).astype("float32")

# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(embeddings)
# print(f"FAISS index built with {index.ntotal} vectors.")
# faiss.write_index(index, "faiss_index.bin")
# print("FAISS index saved to faiss_index.bin")

# if we Have the Index File
index = faiss.read_index("faiss_index.bin")  


In [ ]:
# Prepare The Function to Test The Data with The Prompt For 3 times
def classify_text(query, k=3):
    query_emb = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_emb, k)

    top_examples = [texts[i] for i in indices[0]]
    top_labels = [labels[i] for i in indices[0]]

    context = "\n".join([
        f"Example {i+1}:\nText: {top_examples[i]}\nLabel: {top_labels[i]}"
        for i in range(len(top_examples))
    ])

    prompt = f"""
You are a text classification assistant. 
Based on the following labeled examples, determine the most likely label for the new text.

Labeled examples:
{context}

Now classify this new text:
"{query}"

Respond only with the most appropriate label.
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    model_label = response.choices[0].message.content.strip()

    return model_label


In [ ]:
# Test The RAG on the Number from Data
df = pd.read_excel("test_split.xlsx")
sample_df = df.sample(n=50, random_state=42).reset_index(drop=True)
texts_test = sample_df["text"].astype(str).tolist()
true_labels = sample_df["label"].astype(str).tolist()
predicted_labels = []
for i, query_text in enumerate(texts_test):
    predicted = classify_text(query_text)
    predicted_labels.append(predicted)


accuracy = accuracy_score(true_labels, predicted_labels)
print(f"The Accuracy is :{accuracy:.2f}")
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))

